## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Using GenAI to postion our trucks based upon what's happening in the city

### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- This notebook will use GenAI to determine the best places to locate our coffee trucks in each city based upon the events happening in the city.  The LLM can rank the events from the events that will have the highest impact on sales to the events with the lowest impact.

- Notebook Logic:
    1. Create a row, per city, in the event_gen_ai_insight table.  
        - For each city:
            - Query BigQuery for the current days events.
            - BigQuery will return the results as JSON.
            - Our LLM prompt will be created:
                - The prompt will ask for insights to be created
                - The prompt will pass in knowledge about past event types and the effect on profits
                - The prompt will ask the LLM to compute the latitude and longitude of the event address
                - The prompt will ask the LLM convert date that a human would only undertand to a BigQuery datetime.
                - The prompt will ask to rank the events from 1 to {x}.
                - The prompt will ask the LLM to provide an explaination of its thought process.
                - The prompt will provide example JSON results so the LLM knows the output format to generate the results.
                - The prompt will include the events from BigQuery as the context.

            - A row will be inserted into the event_gen_ai_insight table.

    2. An update statement will be run on the event_gen_ai_insight table which will call GENERATE_TEXT (text-bison) to execute the LLM prompt.  
        - We save our prompt in the table so we can later inspect it or reprocess the prompt.

    3. The results of the GENERATE_TEXT are JSON so the inner JSON is then parsed into the generated_insight_text and generated_insight_json fields.

    4. The data is then displayed.

## Initialize Python

In [ ]:
# No need to set these
city_names=["New York City", "London", "Tokyo", "San Francisco"]
city_ids=[1,2,3,4]
city_timezones=["US/Eastern","Europe/London","Asia/Tokyo","US/Pacific"]
number_of_coffee_trucks = "4"

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

## Create the GenAI Insights table

In [ ]:
%%bigquery
CREATE TABLE IF NOT EXISTS `${project_id}.${bigquery_data_beans_curated_dataset}.event_gen_ai_insight`
--CREATE OR REPLACE TABLE `${project_id}.${bigquery_data_beans_curated_dataset}.event_gen_ai_insight` -- only use this to start over
(
    event_gen_ai_insight_id   INTEGER   NOT NULL OPTIONS(description="Primary key."),
    event_gen_ai_insight_type STRING    NOT NULL OPTIONS(description="The type of insight: WEATHER, EVENT, CAMPAIGN, SOCIAL, WAIT-TIME, SALES, VIDEO-AI"),
    insight_datetime          TIMESTAMP NOT NULL OPTIONS(description="The datetime of the GenAI insight."),
    applies_to_entity_type    STRING    NOT NULL OPTIONS(description="The type of entity to which the insight applies: Company, City, Truck, Customer"),
    applies_to_entity_id      INTEGER   NOT NULL OPTIONS(description="The id (primary key) of entity to which the insight applies."),
    applies_to_entity_name    STRING    NOT NULL OPTIONS(description="The name to which the insight applies."),
    llm_prompt                STRING    NOT NULL OPTIONS(description="The LLM prompt."),
    ml_generate_json_result   JSON               OPTIONS(description="The raw JSON output of the LLM."),
    generated_insight_text    STRING             OPTIONS(description="The generated insight in text"),
    generated_insight_json    JSON               OPTIONS(description="The generated insight in JSON")
)
CLUSTER BY event_gen_ai_insight_id;

In [ ]:
%%bigquery

-- remove the current days events (in case you re-run this notebook)
DELETE
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.event_gen_ai_insight` AS event_gen_ai_insight
 WHERE CAST(insight_datetime AS DATE) = (SELECT CAST(MAX(insight_datetime) AS DATE) FROM `${project_id}.${bigquery_data_beans_curated_dataset}.event_gen_ai_insight`);

## Supporting Functions

In [ ]:
def RunQuery(sql):
  import time

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

In [ ]:
def GetNextPrimaryKey(fully_qualified_table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{fully_qualified_table_name}`
  """
  # print(sql)
  df_result = client.query(sql).to_dataframe()
  # display(df_result)
  return df_result['result'].iloc[0] + 1

## Create the LLM prompt for each city and insert into or GenAI Insights table.

In [ ]:
# Get the next primary key
event_gen_ai_insight_id = GetNextPrimaryKey("${project_id}.${bigquery_data_beans_curated_dataset}.event_gen_ai_insight","event_gen_ai_insight_id")

# Loop for each city
for city_index in range(0, 4):
  print(f"Processing city: {city_ids[city_index]}")

  sql=f"""WITH event_data AS
  (
    SELECT DISTINCT event_id, event_title, event_venue, event_address, event_description
      FROM `${project_id}.${bigquery_data_beans_curated_dataset}.event`
    WHERE city_id = {city_ids[city_index]}
      AND event_date = (SELECT MAX(event_date) FROM `${project_id}.${bigquery_data_beans_curated_dataset}.event`)
  )
  SELECT TO_JSON(t) AS event_json
  FROM event_data AS t
  ORDER BY GENERATE_UUID()
  """

  # Run the SQL to get the events for this city
  print(f"Gathering events for: {city_names[city_index]}")
  event_df = RunQuery(sql)

  # Loop through the dataframe (we want a JSON string for our prompt)
  event_json = ""
  for index, row in event_df.iterrows():
    event_json = event_json + row['event_json'] + "\n"

  # Create the LLM prompt

  # Example return JSON (without the python f"" formatting)
  json_example_data='[ { "rank" : 1, "event_id" : 4, "latitude" : 40.736874, "longitude": -73.985394, ' + \
    '"explanation" : "This will increase traffic since this is a family event." }, ' + \
    '{ "rank" : 2, "event_id" : 6, "latitude" : 41.723124, "longitude": -72.142231, ' + \
    '"explanation" : "This will increase traffic since this is a family event." } ]'

  llm_prompt=f"""You run a fleet of {number_of_coffee_trucks} coffee trucks in {city_names[city_index]} and want to maximize your profits.
  The below is a list of events that are happening tonight.
  Think outside the box and develop a unique and unexpected solution for each location.
  Encourage unconventional ideas and fresh perspectives in your recommendations."  
  Return the results in JSON with no special characters or formatting.
  Use the field "event_id" from the events and place the result in the field "event_id".
  Rank the events from the best to the lowest and place in the field "rank".
  Compute the latitude of the field "event_address" and place the result in the field "latitude".
  Compute the longitude of the field "event_address" and place the result in the field "longitude".
  Explain your logic and place the result in the field "explanation".

  Example Return Data:
  {json_example_data}

  Events:
  {event_json}
  """

  print(f"Created LLM Prompt for: {city_names[city_index]}")

  # Clean illegal SQL characters from Google Events API
  prompt=llm_prompt.replace("\\_","")
  prompt=llm_prompt.replace("\\m","")

  # Insert new row
  sql=f"""INSERT INTO `${project_id}.${bigquery_data_beans_curated_dataset}.event_gen_ai_insight`
  (event_gen_ai_insight_id, event_gen_ai_insight_type, insight_datetime, applies_to_entity_type, applies_to_entity_id, applies_to_entity_name, llm_prompt)
  VALUES({event_gen_ai_insight_id}, 'EVENT', CURRENT_TIMESTAMP(), 'CITY', {city_ids[city_index]}, '{city_names[city_index]}',\"\"\"{prompt}\"\"\")
  """.replace("  ","")

  RunQuery(sql)

  # Get the next primary key
  event_gen_ai_insight_id = event_gen_ai_insight_id + 1

  print(f"Inserted prompt into event_gen_ai_insight with event_gen_ai_insight_id = {event_gen_ai_insight_id}")

## Process the LLM prompt with BigQuery

In [ ]:
# Run the LLM
temperature=.75
max_output_tokens=8192
top_p=.8
top_k=30

sql=f"""UPDATE `${project_id}.${bigquery_data_beans_curated_dataset}.event_gen_ai_insight` AS event_gen_ai_insight
           SET insight_datetime = CURRENT_TIMESTAMP(),
               ml_generate_json_result = llm_query.ml_generate_text_result
           FROM (SELECT *
                   FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_data_beans_curated_dataset}.gemini_pro`,
                       (SELECT event_gen_ai_insight_id,
                               llm_prompt AS prompt
                         FROM `${project_id}.${bigquery_data_beans_curated_dataset}.event_gen_ai_insight`
                         WHERE (ml_generate_json_result IS NULL
                                OR
                                JSON_VALUE(ml_generate_json_result, '$.candidates[0].content') IS NULL
                                )
                       ),
                       STRUCT(
                         {temperature} AS temperature,
                         {max_output_tokens} AS max_output_tokens,
                         {top_p} AS top_p,
                         {top_k} AS top_k
                         ))
           ) AS llm_query
         WHERE event_gen_ai_insight.event_gen_ai_insight_id = llm_query.event_gen_ai_insight_id;
"""
RunQuery(sql)

## Parse and view the results

In [ ]:
%%bigquery

# Parse the LLM results into json fields
UPDATE `${project_id}.${bigquery_data_beans_curated_dataset}.event_gen_ai_insight`
   SET generated_insight_text = `${project_id}.${bigquery_data_beans_curated_dataset}.gemini_pro_result_as_string`(ml_generate_json_result),
       generated_insight_json = `${project_id}.${bigquery_data_beans_curated_dataset}.gemini_pro_result_as_json`(ml_generate_json_result)
 WHERE generated_insight_text IS NULL
 --AND event_gen_ai_insight_id = 3

In [ ]:
%%bigquery

# Display the results
SELECT event.event_id, event.event_date, event.event_title, event.event_venue, event.event_time_string,
       event_gen_ai_insight.applies_to_entity_name, json_data.rank,
       json_data.latitude, json_data.longitude, CAST(JSON_VALUE(json_data.explanation) AS STRING) AS explanation
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.event_gen_ai_insight` AS event_gen_ai_insight
        CROSS JOIN UNNEST(JSON_EXTRACT_ARRAY(event_gen_ai_insight.generated_insight_json)) AS json_data
        INNER JOIN `${project_id}.${bigquery_data_beans_curated_dataset}.event` AS event
                ON CAST(JSON_VALUE(json_data,'$.event_id') AS INT64) = event.event_id
 WHERE event.event_date = (SELECT MAX(event_date) FROM `${project_id}.${bigquery_data_beans_curated_dataset}.event`)
ORDER BY applies_to_entity_name, CAST(JSON_VALUE(json_data.rank) AS INT64);